# Test Fly Abdomens Segmentation and generate Reference Image

This notebook is used to test each step used in the detection and segmentation of the surface of the abdomen in an image stack. Select the segmentation parameters appropriate to your images and save one image as a reference.

In [1]:
# import packages 
from skimage import io, transform
import numpy as np
import open3d as o3d
import copy
import napari
import time
import os
from skimage import morphology
from skimage.measure import label, regionprops, block_reduce
from scipy import stats, ndimage
import matplotlib.pyplot as plt
import sys

root_dir = os.path.join(os.getcwd(), '..')
sys.path.append(root_dir)

from src.preprocess import segmentation_with_optimized_thresh, image_padding, refine_surface_mask, refine_surface_mask_with_mesh_interpolation
from src.aux_pcd_functions import pcd_to_image, image_to_pcd


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


### Set parameters:

In [2]:
# Downscaling of confocal stacks along z,x,y, 
# pick numbers that make the resolution isotropic:
downscaling = (1, 2.5, 2.5) 

# Bit depth of raw images 
# (depending on the detector settings this may be 8, 12 or 16)
bit_depth = 12

# Default values of the parameters used for the segmentation:
segm_pars={'threshold': 1.05, 'max_iter': 200,'fraction_range': [0.04, 0.05], 
           'padding': 20, 'closing_r': 4, 'dilation_r': 4, 'mesh_radius': 10}

### Downscaling and visualization of raw data

In [3]:
# Load the image, only fluorescence channel is required:
reference_fly_filename = "../test_dataset/01_raw/C1-Image1.tif"
image = io.imread(reference_fly_filename)

# Apply downscaling and visualize the stack:
new_image_shape = [int(image.shape[i]/downscaling[i]) for i in range(3)]
image_downscaled = transform.resize(image, new_image_shape, preserve_range = True, anti_aliasing=True)
viewer = napari.view_image(image_downscaled)

### Thresholding:

In [4]:
# Segment:
Thresholded_Image = segmentation_with_optimized_thresh(image_downscaled,
                                                       segm_pars['threshold'], 
                                                       segm_pars['max_iter'], 
                                                       segm_pars['fraction_range'])
viewer = napari.view_image(Thresholded_Image)

## Clean up the segmented volume with morphological transformations:

In [5]:
# Padding:
Mask_padded  = image_padding(Thresholded_Image, padding = segm_pars['padding'])
Image_padded = image_padding(image_downscaled, padding = segm_pars['padding'])

# Clean up the segmentation with morphological transformations:
Refined_Mask = refine_surface_mask(Mask_padded, Image_padded, segm_pars['closing_r'], segm_pars['mesh_radius'])

# Visualize the result:
viewer = napari.view_image(Refined_Mask)

## Transform the segmented volume in a point cloud object and fit a mesh to fill potential holes:

In [11]:
# Define functions to visualize the point cloud object:

def custom_draw(pcd):
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    opt = vis.get_render_option()
    opt.background_color = np.asarray([0, 0, 0])
    vis.add_geometry(pcd)
    vis.run()
    vis.destroy_window()
    
def custom_draw_w_mesh(pcd, mesh):
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    opt = vis.get_render_option()
    opt.background_color = np.asarray([0, 0, 0])
    vis.add_geometry(pcd)
    vis.add_geometry(mesh)
    vis.run()
    vis.destroy_window()
    
def draw_pcd(pcd):
    pcd_temp = copy.deepcopy(pcd)
    pcd_temp.paint_uniform_color([1, 0.706, 0])
    o3d.visualization.draw_geometries([pcd_temp])


# Transform the mask surface in a point cloud and interpolate it with a mesh:
mesh_pcd, mesh_pcd_values, original_pcd, mesh = refine_surface_mask_with_mesh_interpolation(
                                        Refined_Mask, segm_pars['mesh_radius'])

custom_draw_w_mesh(original_pcd, mesh)
custom_draw(mesh_pcd)

## Create a mask from the mesh and apply it to the original image stack:

In [13]:
final_image = pcd_to_image(mesh_pcd, mesh_pcd_values, Refined_Mask.shape)
final_image = morphology.dilation(final_image, morphology.ball(segm_pars['dilation_r']))
viewer = napari.view_image(final_image*Image_padded)

INFO - 2023-06-21 17:32:08,397 - acceleratesupport - No OpenGL_accelerate module loaded: No module named 'OpenGL_accelerate'


## Save the masked image

In [ ]:
# Save it as reference
from tifffile import imsave
imsave("../../data_2/References_and_masks/C1_Reference_iso.tiff", (final_image*Image_padded).astype(np.uint16))